In [1]:
import sys
sys.path[0]+='/..'
from dataset import GraphDataset
import torch
import torch_geometric
import os 
from model.gcn import GNNPolicy
import time
from torch.optim.lr_scheduler import CosineAnnealingLR

In [2]:
#set folder

model_save_path = '../model_save/1bin_10_pretrain_routine/'
log_save_path = "../model_save/1bin_10_log_routine/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)

log_file = open(f'{log_save_path}train.log', 'wb')

In [3]:
LEARNING_RATE = 1e-4
NB_EPOCHS =2000
BATCH_SIZE = 64
NUM_WORKERS = 2
ETA_MIN=1e-5

In [4]:
train_data = GraphDataset('../datasets/10_std/1bin/train/')
valid_data = GraphDataset('../datasets/10_std/1bin/valid/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
PredictModel = GNNPolicy().to(DEVICE)
# if os.path.exists(model_save_path+'model_best.pth'):
#     state_dict = torch.load(model_save_path+'model_best.pth')
#     PredictModel.load_state_dict(state_dict)
#     print("载入模型成功")


In [7]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.weighted_sol
            # print(len(batch.variable_features[torch.isnan(batch.variable_features)]))
            #predict the binary distribution, BD
            BD = predict(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
            )
            # compute loss
            
            presol = BD[batch.b_vars==1]
            presol = presol.sigmoid()
            loss = loss_fn(presol, weighted_sol)
            

            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed

    return mean_loss


In [8]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS, eta_min=ETA_MIN)
best_val_loss = 0.06

In [9]:
cur_epoch = 0
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.6f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.6f}")
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print(best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    #100个epochloss不下降就停止
    if cur_epoch>300 and valid_loss>best_val_loss:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 0.646261	Epoch 0 Valid loss: 0.565819
Epoch 1 Train loss: 0.437733	Epoch 1 Valid loss: 0.308650
Epoch 2 Train loss: 0.228648	Epoch 2 Valid loss: 0.175392
Epoch 3 Train loss: 0.163154	Epoch 3 Valid loss: 0.150597
Epoch 4 Train loss: 0.149648	Epoch 4 Valid loss: 0.146801
Epoch 5 Train loss: 0.146991	Epoch 5 Valid loss: 0.148807
Epoch 6 Train loss: 0.146185	Epoch 6 Valid loss: 0.141782
Epoch 7 Train loss: 0.140977	Epoch 7 Valid loss: 0.137649
Epoch 8 Train loss: 0.136450	Epoch 8 Valid loss: 0.134998
Epoch 9 Train loss: 0.130521	Epoch 9 Valid loss: 0.132007
Epoch 10 Train loss: 0.128630	Epoch 10 Valid loss: 0.135167
Epoch 11 Train loss: 0.127891	Epoch 11 Valid loss: 0.128221
Epoch 12 Train loss: 0.126897	Epoch 12 Valid loss: 0.130163
Epoch 13 Train loss: 0.125584	Epoch 13 Valid loss: 0.131410
Epoch 14 Train loss: 0.126560	Epoch 14 Valid loss: 0.128602
Epoch 15 Train loss: 0.126414	Epoch 15 Valid loss: 0.128383
Epoch 16 Train loss: 0.124851	Epoch 16 Valid loss: 0.126202
